In [4]:
import pandas as pd
import numpy as np
from datetime import date

import lifetimes
import pandas_profiling as pp

import numpy as np
import sklearn

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [5]:
pd.options.mode.chained_assignment = None

In [3]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

## 데이터 설명

주어진 데이터는, 사용자들의 구매내역 데이터로, 
<br/>
날짜
<br/>
상품id
<br/>
구매id > agg해서 봐야한다
<br/>
payment code
<br/>
patment name
<br/>
취소여부
<br/>
회원DE
<br/>
단말기
<br/>
고객의 실제 지불금액
<br/>
쿠폰 사용금액
<br/>
nan set 무시
<br/>
구매 시간 분류
<br/>
성별
<br/>
연력
<br/>
이동통신사 코드 
<br/>
카테고리 넘버 
<br/>
카테고리 이름(정액제>e북이 북패스), 
<br/>
상세카테고리
<br/>
세부분류
<br/>
상품 원가
<br/>
구독코드
<br/>
패키지이름
<br/>
앱플레이어
<br/>

(1) if 현금, 쿠폰, 캐시를 했으면 세개의 건수로 잡히는데 구매id로 묶어서 봐야한다 
<br/>
(2) cancel Y/N Y값이 있으면 해당되는 중복을 지워야한다 
<br/>
(3) partition_dt 앞쪽은 날짜, purcha tm c ~ 시기구간별:뒤쪽은 구매시간
<br/>
(4) 북패스 구매자 : 상품 끝자리 635, 카테고리dpff, 

In [4]:
data1 = pd.read_csv('P:/ord_prchs_diet.csv')

In [6]:
data1.shape

(40131396, 14)

In [7]:
data1.head()

,partition_dt,prchs_id,payment_mtd_nm,insd_usermbr_no,cust_payment_amt,sett_target_cpn_amt,prchs_tm_clsf_nm,sex_clsf_cd,age_cd,mno_cd,category_nm,dtl_category_nm,prod_amt,prod_grd_cd
0,20200201,20020119474421413237,ONE pay 카드,US2015062611394912110641133,2600,0,19 ~ 20,F,044,US001202,eBook,로맨스,4000,PD004404
1,20200201,20020120075710113540,컬쳐캐쉬,US2016060616574832325791427,2430,0,20 ~ 21,F,026,US001201,eBook,로맨스,2700,PD004404
2,20200201,20020111240130710381,북스 캐쉬 (현금),US2019082620122445453051413,2640,0,11 ~ 12,M,066,US001203,eBook,로맨스,2700,PD004404
3,20200201,20020111240130710381,북스 캐쉬 (포인트),US2019082620122445453051413,0,60,11 ~ 12,M,066,US001203,eBook,로맨스,2700,PD004404
4,20200201,20020110204410231507,북스 캐쉬 (현금),IW1424469760220110528222309,2700,0,10 ~ 11,F,045,US001201,eBook,로맨스,2700,PD004404


## 불필요한 column 삭제

In [12]:
data1 = data1.drop(['prod_grd_cd'], axis = 1)

In [14]:
# a = pd.DataFrame(data1.groupby(['prchs_id', 'prod_id'])['payment_mtd_nm'].count()).reset_index()
# a

In [55]:
data1 = pd.merge(data1, a, how = 'left', on = ['prchs_id', 'prod_id'])

In [56]:
data1= data1.drop

,partition_dt,prod_id,prchs_id,payment_mtd_nm_x,insd_usermbr_no,cust_payment_amt,sett_target_cpn_amt,prchs_tm_clsf_nm,sex_clsf_cd,age_cd,mno_cd,category_nm,dtl_category_nm,prod_amt,payment_mtd_nm_y
0,20200315,H037784296,20031517045411126112,ONE pay 카드,IM142100007817868201305101526,360,0,17 ~ 18,F,047,US001201,웹툰,BL,400,1
1,20200315,H037789246,20031519204711539321,네이버페이,US2017040422011068736556784,90,0,19 ~ 20,Z,ZZZ,US001201,eBook,무협,100,2
2,20200315,H037789246,20031519204711539321,T 멤버십,US2017040422011068736556784,0,10,19 ~ 20,Z,ZZZ,US001201,eBook,무협,100,2
3,20200315,H037819161,20031513470510711172,휴대폰결제 (DCB),US201502190828539284589824,100,0,13 ~ 14,M,051,US001201,eBook,판타지,100,1
4,20200315,H037819161,20031505511710810621,북스 캐쉬 (현금),US2018030620352888244110712,100,0,05 ~ 06,M,038,US001201,eBook,판타지,100,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416660,20200507,H039079540,20050714193020711329,북스 캐쉬 (현금),US2019111707235842454181616,100,0,14 ~ 15,Z,ZZZ,US001202,eBook,로맨스,100,1
416661,20200507,H039079991,20050709090031519466,북스 캐쉬 (현금),US2017032923021420836418799,100,0,09 ~ 10,M,050,US001203,eBook,판타지,100,1
416662,20200507,H039080656,20050718191611530994,북스 캐쉬 (현금),IW1424516471220110606010606,100,0,18 ~ 19,F,042,US001201,eBook,로맨스,100,1
416663,20200507,H039081547,20050701174510536154,북스 캐쉬 (포인트),US2016120917563137233351253,0,200,01 ~ 02,F,038,US001201,만화,순정,200,1


In [49]:
data1.shape

(416665, 16)

## 데이터 둘러보기 

In [6]:
data = pd.read_csv('P:/final_prchs_data.csv')

In [7]:
data.head()

,prchs_id,prod_id,partition_dt,insd_usermbr_no,prod_amt,sex_clsf_cd,age_cd,category_nm,dtl_category_nm,prchs_tm_clsf_nm,mno_cd,cust_payment_amt,sett_target_cpn_amt,pay_mtd_cnt
0,20013123592010721507,H018480692,20200201,IF1423177586220101006224941,3600,F,042,eBook,로맨스,00 ~ 01,US001201,3240,360,3
1,20013123593010931584,H038835416,20200201,US2018070914480184346187029,900,M,037,eBook,무협,00 ~ 01,US001201,810,90,2
2,20013123593510331626,H031234584,20200201,US2018092811402674047654410,900,Z,ZZZ,eBook,판타지,00 ~ 01,US001201,810,90,2
3,20013123595120417183,H038724908,20200201,US2018033113065860244558735,3150,F,051,eBook,로맨스,00 ~ 01,US001202,3150,0,1
4,20013123595221217187,H035730852,20200201,US2019112415415699154280728,200,F,043,웹툰,순정,00 ~ 01,US001202,180,20,2


## 날짜 변수 형식 변환 및 추가

In [8]:
data.dtypes

prchs_id               object
prod_id                object
partition_dt            int64
insd_usermbr_no        object
prod_amt                int64
sex_clsf_cd            object
age_cd                 object
category_nm            object
dtl_category_nm        object
prchs_tm_clsf_nm       object
mno_cd                 object
cust_payment_amt        int64
sett_target_cpn_amt     int64
pay_mtd_cnt             int64
dtype: object

data['partition_dt']이 datatime 형식으로 되어있지 않으므로, 바꾸어준다. 
<br/>
또 EDA를 위하여 몇 가지 날짜 변수를 추가한다. > 우선 월과 요일

In [9]:
import datetime
data['partition_dt'] = pd.to_datetime(data['partition_dt'], format='%Y%m%d')

In [10]:
# data['month'] = data['partition_dt'].dt.month
# data['weekday'] = data['partition_dt'].dt.weekday

# 데이터 둘러보기 

In [ ]:
display(data['category_nm'].unique(),data['category_nm'].value_counts())

In [ ]:
data['dtl_category_nm'].unique()

In [ ]:
data['dtl_category_nm'].value_counts()

## Small set test for 공휴일

smaller example

In [11]:
temp11 = ["2020-03-01", "2020-04-15", "2020-04-30", "2020-05-05", "2020-06-06"]

In [12]:
data11 = [["a", "2020-03-14"], ["b","2020-03-18"], ["c", "2020-06-06"], ['d', '2020-04-30']]
df = pd.DataFrame(data11, columns = ['name', 'datee'])
df["holiday"] = ""

In [13]:
# for i in range(4):
#     if df['datee'][i] in temp11:
#         df["holiday"][i] = True
#     else:
#         df["holiday"][i] = False

In [14]:
df['holiday'] = df.apply(lambda x : 1 if x['datee'] in temp11 else 0, axis = 1)

In [15]:
df

,name,datee,holiday
0,a,2020-03-14,0
1,b,2020-03-18,0
2,c,2020-06-06,1
3,d,2020-04-30,1


In [16]:
# data['holiday'] = np.where(data['holiday'] in temp, 1, 0)
# 왜 안되는걸까

## 공휴일 feature 추가

현재 데이터에는 공휴일 구매 데이터가 없는 것 같으므로(?) 위 코드로 확인

In [17]:
data["holiday"] = ""

In [18]:
data['holiday'] = data.apply(lambda x: 1 if x['partition_dt'] in temp else 0, axis = 1)

MemoryError: Unable to allocate 261. MiB for an array with shape (34246920,) and data type int64

In [ ]:
for i in range(416665):
    if data['partition_dt'][i] in temp11:
        data['holiday'][i] = True
    else:
        data['holiday'][i] = False

In [ ]:
pd.DataFrame(data['holiday'])

In [ ]:
pd.DataFrame(data.groupby('prchs_id')['holiday'].sum())

주어진 2020년 데이터에 공휴일이 없으므로, 추가가 무의미하다.